In [1]:
import os
import pandas as pd
from PIL import Image
import torch
from torch.utils.data import Dataset
from torchvision import transforms


In [2]:
class ChestXrayDataset(Dataset):
    def __init__(self, csv_file, image_dir, transform=None):
        self.data = pd.read_csv(csv_file)
        self.image_dir = image_dir
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_name = self.data.iloc[idx, 0]
        label = self.data.iloc[idx, 5]

        img_path = os.path.join(self.image_dir, img_name)
        image = Image.open(img_path).convert("RGB")

        if self.transform:
            image = self.transform(image)

        return image, torch.tensor(label, dtype=torch.long)


In [3]:
train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

test_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

In [4]:
from torch.utils.data import DataLoader

img_dir = r"C:\Users\sureb\my_data\images\primary_data"
img_dir1 = r"C:\Users\sureb\my_data\images\secondary_data"

train_dataset = ChestXrayDataset(
    csv_file=r"C:\Users\sureb\my_data\primary_data.csv",
    image_dir=img_dir,
    transform=train_transform
)

test_dataset = ChestXrayDataset(
    csv_file=r"C:\Users\sureb\my_data\secondary_data.csv",
    image_dir=img_dir1,
    transform=test_transform
)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)


In [5]:
import torch.nn as nn
import torch.nn.functional as F


In [6]:
class ChestXrayCNN(nn.Module):
    def __init__(self, num_classes=3):
        super().__init__()

        self.features = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(2),

            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(2),

            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(2),

            nn.Conv2d(128, 256, kernel_size=3, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.AdaptiveAvgPool2d((1, 1))
        )

        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(128, num_classes)
        )

    def forward(self, x):
        x = self.features(x)
        x = self.classifier(x)
        return x

In [7]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = ChestXrayCNN(num_classes=3).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

In [8]:
def train_one_epoch(model, loader, optimizer, criterion, device):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for images, labels in loader:
        images = images.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

        _, preds = torch.max(outputs, 1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)

    avg_loss = running_loss / len(loader)
    accuracy = 100.0 * correct / total

    return avg_loss, accuracy


In [9]:
def evaluate(model, loader, criterion, device):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        for images, labels in loader:
            images = images.to(device)
            labels = labels.to(device)

            outputs = model(images)
            loss = criterion(outputs, labels)

            running_loss += loss.item()

            _, preds = torch.max(outputs, 1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)

    avg_loss = running_loss / len(loader)
    accuracy = 100.0 * correct / total

    return avg_loss, accuracy


In [10]:
num_epochs = 30

for epoch in range(num_epochs):
    train_loss, train_acc = train_one_epoch(
        model, train_loader, optimizer, criterion, device
    )

    test_loss, test_acc = evaluate(
        model, test_loader, criterion, device
    )

    print(
        f"Epoch [{epoch+1}/{num_epochs}] | "
        f"Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.2f}% | "
        f"Test Loss: {test_loss:.4f}, Test Acc: {test_acc:.2f}%"
    )


Epoch [1/30] | Train Loss: 1.0824, Train Acc: 40.95% | Test Loss: 1.0926, Test Acc: 51.58%
Epoch [2/30] | Train Loss: 0.9696, Train Acc: 54.29% | Test Loss: 1.0910, Test Acc: 52.49%
Epoch [3/30] | Train Loss: 0.9380, Train Acc: 57.62% | Test Loss: 1.0943, Test Acc: 29.41%
Epoch [4/30] | Train Loss: 0.9389, Train Acc: 55.71% | Test Loss: 1.1077, Test Acc: 28.96%
Epoch [5/30] | Train Loss: 0.9094, Train Acc: 54.76% | Test Loss: 1.1355, Test Acc: 28.96%
Epoch [6/30] | Train Loss: 0.8837, Train Acc: 58.10% | Test Loss: 1.1671, Test Acc: 28.96%
Epoch [7/30] | Train Loss: 0.8912, Train Acc: 55.24% | Test Loss: 1.1647, Test Acc: 30.32%
Epoch [8/30] | Train Loss: 0.8692, Train Acc: 59.52% | Test Loss: 1.1565, Test Acc: 28.96%
Epoch [9/30] | Train Loss: 0.8638, Train Acc: 61.43% | Test Loss: 1.1735, Test Acc: 28.96%
Epoch [10/30] | Train Loss: 0.8636, Train Acc: 56.19% | Test Loss: 1.1882, Test Acc: 28.96%
Epoch [11/30] | Train Loss: 0.8437, Train Acc: 59.52% | Test Loss: 1.2020, Test Acc: 28.9